In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [3]:
# del session.logger
# del session

from pickle import NONE
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "schoffelen2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 40)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/ablation/downsample200/40",
    clear_cache=True,
    cache_name="cache",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Cleared cache for study gwilliams2023
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


Training Epoch 1: 100%|██████████| 135/135 [45:27<00:00, 20.20s/it] 


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0880, Top 1: 0.1420, Top 5: 0.3580, Top 10: 0.4925, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0759, Top 1: 0.1144, Top 5: 0.3601, Top 10: 0.5292, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0688, Top 1: 0.1187, Top 5: 0.3375, Top 10: 0.4888, Perplexity: 0.0000
Epoch 1 completed in 52.14m. 0.39m per recording.


Training Epoch 2: 100%|██████████| 135/135 [05:15<00:00,  2.33s/it]


Testing at epoch 2
Test unseen_subject completed. Accuracy: 0.0495, Top 1: 0.0955, Top 5: 0.2875, Top 10: 0.4170, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0598, Top 1: 0.0881, Top 5: 0.2791, Top 10: 0.4233, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0537, Top 1: 0.0900, Top 5: 0.2662, Top 10: 0.4100, Perplexity: 0.0000
Epoch 2 completed in 5.70m. 0.04m per recording.


...

Testing at epoch 23
Test unseen_subject completed. Accuracy: 0.1490, Top 1: 0.2300, Top 5: 0.4675, Top 10: 0.5780, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1659, Top 1: 0.2266, Top 5: 0.5155, Top 10: 0.6591, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1325, Top 1: 0.1925, Top 5: 0.4337, Top 10: 0.5575, Perplexity: 0.0000
Epoch 23 completed in 6.22m. 0.05m per recording.


Training Epoch 24: 100%|██████████| 135/135 [05:47<00:00,  2.57s/it]


Testing at epoch 24
Test unseen_subject completed. Accuracy: 0.1430, Top 1: 0.2150, Top 5: 0.4540, Top 10: 0.5615, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1162, Top 1: 0.1674, Top 5: 0.4269, Top 10: 0.5758, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1087, Top 1: 0.1550, Top 5: 0.3488, Top 10: 0.4675, Perplexity: 0.0000
Epoch 24 completed in 6.31m. 0.05m per recording.


Training Epoch 25: 100%|██████████| 135/135 [05:49<00:00,  2.59s/it]


Testing at epoch 25
Test unseen_subject completed. Accuracy: 0.1435, Top 1: 0.2145, Top 5: 0.4555, Top 10: 0.5640, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1550, Top 1: 0.2116, Top 5: 0.4908, Top 10: 0.6285, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1175, Top 1: 0.1775, Top 5: 0.3813, Top 10: 0.5200, Perplexity: 0.0000
Epoch 25 completed in 6.31m. 0.05m per recording.
Early stopping at epoch 25. Highest top 10 accuracy at epoch 14.
Training completed.
unseen_subject: Acc: 0.1850, Top 1: 0.2650, Top 5: 0.5265, Top 10: 0.6515
unseen_task: Acc: 0.2232, Top 1: 0.2973, Top 5: 0.6093, Top 10: 0.7418
unseen_both: Acc: 0.1900, Top 1: 0.2675, Top 5: 0.5013, Top 10: 0.6338


## 200Hz, 40